In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#Creating New folders in the directory

In [0]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Example
createFolder('/content/gdrive/My Drive/Exact_space/new/asset_1/')
createFolder('/content/gdrive/My Drive/Exact_space/new/asset_2/')

#Extracting and Saving Video Frames using OpenCV-PythonPython

In [0]:
#Extracting and Saving Video Frames using OpenCV-PythonPython for asset 1
import cv2 
# Opens the Video file
cap= cv2.VideoCapture('/content/gdrive/My Drive/Exact_space/videos/class_1.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/gdrive/My Drive/Exact_space/new/asset_1/class_1_'+str(i)+'.jpg',frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

In [0]:
#Extracting and Saving Video Frames using OpenCV-PythonPython for asset 2
import cv2 
# Opens the Video file
cap= cv2.VideoCapture('/content/gdrive/My Drive/Exact_space/videos/class_2.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/gdrive/My Drive/Exact_space/new/asset_2/class_2_'+str(i)+'.jpg',frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

#Image Augmentation

Random crop genarator for cropping the images

Applying drop in the images -Sometimes there could be drop in signal

Introducing noise -Incase if there is any disturbance in the Signal(mask)

Blurring - If the video capture is not good or not clear  (dust,moisture,mist....)

Introducing noise -Incase if there is any disturbance in the Signal(gaussian noise) -random noise

In [0]:
#Random crop genarator for cropping the images
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)
        


#Importing imgaug 

from imgaug import augmenters as iaa

#Applying drop in the images -Sometimes there could be drop in signal
def apply_drop(batches):
  
  while True:
    batch_x, batch_y = next(batches)
    batch_drop = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], batch_x.shape[3])) 
    
    for i in range(batch_x.shape[0]):
        image_array = np.asarray(batch_x[i])
        drop = iaa.CoarseDropout((0.0, 0.5), size_percent=(0.2, 0.5))
        batch_drop[i] = drop.augment_image(image_array)
    yield (batch_drop, batch_y)
    
    
    
# Introducing noise -Incase if there is any disturbance in the Signal(mask)
import random

def noising(image):
    array = np.array(image)
    i = random.choice(range(16,48)) # x coordinate for the top left corner of the mask
    j = random.choice(range(16,48)) # y coordinate for the top left corner of the mask
    array[i:i+16, j:j+16]=-1.0 # setting the pixels in the masked region to -1
    return array
def noise_generator(batches):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_noise = np.zeros((batch_x.shape[0], 64, 64, 3))
        for i in range(batch_x.shape[0]):
            batch_noise[i] = noising(batch_x[i])
        yield (batch_noise, batch_y)
        
        
#Blurring -If the video capture is not good or not clear       
def blur_crop_flip_image(batches, blur_value, crop_value, flip_value):
  seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, blur_value)), # ex: 0.4
    iaa.Crop(percent=(0, crop_value)), # ex: 0.2
    iaa.Sometimes(0.3, iaa.Fliplr(flip_value))]) # 50% flip / horizontal flip of only 30% of the images passed
  while True:
    batch_x, batch_y = next(batches)
    batch_augmented = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], NUM_CHANNELS)) 
    # NOTE: imgaug works on color images (3 channels). doesn't work on greyscale images with one channel
    batch_augmented = seq.augment_images(batch_x) # calling ImgAug's augmentation on a batch of images
    yield (batch_augmented, batch_y)
    

    
    
#Introducing noise -Incase if there is any disturbance in the Signal(gaussian noise)
from skimage.util import random_noise

## Applying Salt and Pepper noise in images.

def apply_noise(batches):
  
  while True:
    batch_x, batch_y = next(batches)
    batch_noise = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], batch_x.shape[3])) 
    
    for i in range(batch_x.shape[0]):
        image_array = np.asarray(batch_x[i])
        batch_noise[i] = random_noise(image_array , mode='gaussian', var =0.01)
    yield (batch_noise, batch_y)
    
# train_generator = apply_noise(train_generator)

###Zoom out, Horizontal shift, Vertical shift, Horizontal flip

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen1 = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  fill_mode='nearest',
                                  horizontal_flip=True)

train_datagen2 = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  fill_mode='nearest',
                                  zoom_range = (2,2),
                                  horizontal_flip=True)

train_datagen3 = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  fill_mode='nearest',
                                  width_shift_range=0.2,
                                  horizontal_flip=True)
train_datagen4 = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  fill_mode='nearest',
                                  height_shift_range=0.2,
                                  horizontal_flip=True)






tr_1 = train_datagen1.flow_from_directory('/content/gdrive/My Drive/Exact_space/exact_space/train/',
                                           target_size=(256, 256),
                                           batch_size=32,
                                           shuffle=True,
                                           class_mode='categorical')
tr_2 = train_datagen2.flow_from_directory('/content/gdrive/My Drive/Exact_space/exact_space/train/',
                                           target_size=(256, 256),
                                           batch_size=32,
                                           shuffle=True,
                                           class_mode='categorical')
tr_3 = train_datagen3.flow_from_directory('/content/gdrive/My Drive/Exact_space/exact_space/train/',
                                           target_size=(256, 256),
                                           batch_size=32,
                                           shuffle=True,
                                           class_mode='categorical')
tr_4 = train_datagen4.flow_from_directory('/content/gdrive/My Drive/Exact_space/exact_space/train/',
                                           target_size=(256, 256),
                                           batch_size=32,
                                           shuffle=True,
                                           class_mode='categorical')

Using TensorFlow backend.


Found 858 images belonging to 2 classes.
Found 858 images belonging to 2 classes.
Found 858 images belonging to 2 classes.
Found 858 images belonging to 2 classes.


###The videos are of different time lengths .

###Inorder to have balanced classes , I took 429 images from each class for training and 5 from each class for testing . 

In [0]:
tr_5 = blur_crop_flip_image(tr_4, 0.3, 0.3, 230)
tr_6= (apply_noise(tr_4))
tr_7 = crop_generator(tr_2,230)
tr_8 = crop_generator(apply_noise(tr_1),230)

tr_9 = noise_generator(tr_2)
tr_10 = apply_drop(tr_3)



from itertools import chain
train_datagen = chain(tr_1,tr_2,tr_3,tr_4,tr_5,tr_6,tr_7,tr_8,tr_9,tr_10)

###The actual images resolutions are 640x352
###And we are reducing it to 256x256(occupies less memory)

In [6]:

test_datagen = ImageDataGenerator(rescale=1./255
                                 )


test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/Exact_space/exact_space/test/',
                                            target_size=(256, 256),
                                            batch_size=8,
                                            shuffle=True,
                                            class_mode='categorical')


Found 10 images belonging to 2 classes.


In [0]:
#importing modules
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import  Add
from keras.layers import Convolution2D, MaxPooling2D,Flatten,SeparableConv2D
from keras.utils import np_utils
import tensorflow as tf
from keras.layers import Activation
from keras.layers import MaxPooling2D

from keras.layers import DepthwiseConv2D

from keras.layers import SeparableConv2D

from keras import regularizers



#Simple image classification model

As the images are easy to classify

In [16]:
model = Sequential()
model.add(SeparableConv2D(16,3, 3, activation='relu', input_shape=(256,256,3)))
model.add(Convolution2D(10, 1, activation='relu'))
model.add(SeparableConv2D(32,3,3,activation='relu'))
model.add(Convolution2D(10, 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(SeparableConv2D(32,3,3,activation='relu'))
model.add(Convolution2D(10, 1, activation='relu'))
model.add(SeparableConv2D(2,124))
model.add(Flatten())
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(16, (3, 3), activation="relu", input_shape=(256, 256,...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (3, 3), activation="relu")`
  import sys


In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_1 (Separabl (None, 254, 254, 16)      91        
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 254, 254, 10)      170       
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 252, 252, 32)      442       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 252, 252, 10)      330       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 124, 124, 32)      442       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 124, 124, 10)      330       
__________

In [19]:
from keras import optimizers


model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


model.fit_generator(train_datagen, epochs=10, steps_per_epoch=8, validation_steps=3,
                    shuffle=True,
                    validation_data=test_set)

Epoch 1/10
8/8 [==============================] - 5s 668ms/step - loss: 0.6832 - acc: 0.5807 - val_loss: 0.6945 - val_acc: 0.4167
Epoch 2/10
8/8 [==============================] - 5s 633ms/step - loss: 0.5959 - acc: 0.7617 - val_loss: 0.4844 - val_acc: 1.0000
Epoch 3/10
8/8 [==============================] - 5s 644ms/step - loss: 0.3261 - acc: 1.0000 - val_loss: 0.1830 - val_acc: 1.0000
Epoch 4/10
8/8 [==============================] - 5s 638ms/step - loss: 0.0760 - acc: 1.0000 - val_loss: 0.0381 - val_acc: 1.0000
Epoch 5/10
8/8 [==============================] - 5s 641ms/step - loss: 0.0102 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 6/10
8/8 [==============================] - 5s 650ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 6.9782e-04 - val_acc: 1.0000
Epoch 7/10
8/8 [==============================] - 5s 646ms/step - loss: 4.3068e-04 - acc: 1.0000 - val_loss: 5.6010e-04 - val_acc: 1.0000
Epoch 8/10
8/8 [==============================] - 5s 651ms/step - loss: 3.0769

In [38]:
# scores = model.evaluate_generator(test_set,10) #1514 testing images
# print("Accuracy = ", scores[1])


pred=model.predict_generator(test_set,steps=2,verbose=1)

2/2 [==============================] - 0s 42ms/step


In [39]:
pred

array([[8.9059693e-05, 9.9991095e-01],
       [9.9979466e-01, 2.0537726e-04],
       [5.8681180e-05, 9.9994135e-01],
       [9.9979347e-01, 2.0652779e-04],
       [3.7994505e-05, 9.9996197e-01],
       [9.7783632e-05, 9.9990225e-01],
       [9.9979466e-01, 2.0537726e-04],
       [9.9981636e-01, 1.8366578e-04],
       [9.9978644e-01, 2.1363400e-04],
       [9.9980205e-01, 1.9801801e-04]], dtype=float32)

In [0]:

predicted_class_indices=np.argmax(pred,axis=1)
labels = (test_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [42]:
predictions

['asset_2',
 'asset_1',
 'asset_2',
 'asset_1',
 'asset_2',
 'asset_2',
 'asset_1',
 'asset_1',
 'asset_1',
 'asset_1']